# Energy Materials: Design, Discovery and Data

## 5. Exploring the Materials Hyperspace

## Advance Reading
[High-throughput Screening of All Stoichiometric Inorganic Materials](opus.bath.ac.uk/52755/) (Chem, 2016)   
[The Scale of Materials Design](https://hackingmaterials.com/2013/11/14/the-scale-of-materials-design/) (Blog entry, Anubhav Jain, 2013)

## Lecture slides



## Contents

- [The Combinatorial Explosion](#combinatorial_explosion)
- [Mapping Out Chemical Space](#smact)
- [Interacting With Databases](#MP_API)

## The Combinatorial Explosion <a id="combinatorial_explosion"></a>

We can use some simple maths plus our Python skills to solve this combinatorial problem.   

<div class="alert alert-success"> 1. First, set some variables to use in our equations

In [ ]:
n_elements =
n_atoms =
n_gridpoints =

<div class="alert alert-success"> 2. Show that assigning each of the $30$ atoms as one of $50$ elements results in about $ 9\times10^{50}$ permutations   
Use the expression for calculating permutations, $n^r$, where $n$ = number of elements and $r$ = number of atoms  

In [ ]:
element_assignments =
print('number of possible element assignments is:  {:.2E}'.format(element_assignments))

<div class="alert alert-success"> 3. Show that the number of possible arrangements of $30$ atoms on a grid of $10\times10\times10$ is $~2\times10^{57}$    
Use the expression for calculating combinations, $\dfrac{n}{r!(n-r)!}$, where $n$ = number of grid points and $r$ = number of atoms <br/>

In [ ]:
from math import factorial as factorial # We import this useful function to make life easier. e.g. 5! = factorial(5)
atom_arrangements =
print('number of atom arrangements is:  {:.2E}'.format(atom_arrangements))

<div class="alert alert-success"> 4. Finally, show that the total number of potential "materials" is about $2\times10^{108}$     
 We require a simple equation which gives us the total number of possibilities from the above results <br/>

In [ ]:
total =
print('total number of "materials" is:  {:.2E}'.format(total))

## Mapping Out Chemical Space <a id="smact"></a>
#### Using SMACT

<div class="alert alert-success"> 1. Add a line to the code below which will print out the last element in the list <br/>

In [ ]:
import smact # For generating Element objects and combine them according to certain rules

# We set up a dictionary of SMACT Element objects
all_elements = smact.element_dictionary()

# We choose the elements we want using another SMACT function and put them in a list
elements = []
for i in smact.ordered_elements(1,18):
    elements.append(all_elements[i])

print('The first element in the list is {0}'.format(elements[0].symbol))
print('The number of elements is {0}'.format(len(elements)))

Let's cycle through all the unique binary combinations in our list:

In [ ]:
for i, ele_a in enumerate(elements):
    for j, ele_b in enumerate(elements[i+1:]):
        print(ele_a.symbol, ele_b.symbol)

Now let's include each oxidation state:

In [ ]:
for i, ele_a in enumerate(elements):
    for ox_a in ele_a.oxidation_states:
        for j, ele_b in enumerate(elements[i+1:]):
            for ox_b in ele_b.oxidation_states:
                print(ele_a.symbol, ox_a, ele_b.symbol, ox_b)

It looks like He, Ne and Ar have disappeared altogether! That's because they have no non-zero oxidation states so SMACT does not include them.

We can add a counter to see how many combinations we have:

In [ ]:
counter = 0
for i, ele_a in enumerate(elements):
    for ox_a in ele_a.oxidation_states:
        for j, ele_b in enumerate(elements[i+1:]):
            for ox_b in ele_b.oxidation_states:
                counter += 1
print('The number of combinations is:  {0}'.format(counter))      

<div class="alert alert-success"> 2. a) Modify the cell above, adding another level of nested for loops, to count up how many _ternary_ combinations are possible for these elements

b) Check that the right thing is being counted by printing out each combination <br/>

Finally, we can add a function so that only allows charge neutral combinations are counted:

In [ ]:
counter = 0
for i, ele_a in enumerate(elements):
    for ox_a in ele_a.oxidation_states:
        for j, ele_b in enumerate(elements[i+1:]):
            for ox_b in ele_b.oxidation_states:
                cn_e, cn_r = smact.neutral_ratios([ox_a, ox_b], threshold=1)
                if cn_e:
                    print('{0} {1} and {2} {3}  are charge neutral in the ratios  {4}'.format(ele_a.symbol, ox_a,
                                                                                              ele_b.symbol, ox_b,
                                                                                              cn_r))
                    counter += 1
print('The number of combinations is:  {0}'.format(counter))

## Interacting with Databases <a id="MP_API"></a>


<div class="alert alert-success"> 1. Insert your own API key in the correct place in the cell below. <br/>
This can be found on your [Materials Project Dashboard](https://materialsproject.org/dashboard).   
You can put the string directly into the parentheses after `MPRester`.

In [ ]:
from pymatgen import MPRester
m = MPRester("MP_API_KEY_GOES_IN_THESE_QUOTE_MARKS")

<div class="alert alert-success"> 2. Check that the API is working by running the cell below. <br/>
Feel free to replace the formula with one of your favourites!

In [ ]:
favourite_formula = 'SnS'

criteria = {'pretty_formula':favourite_formula}  # We set the criteria we want to search for 
properties = ['task_id', 'pretty_formula', 'final_energy_per_atom', 'full_formula']  # and what properties we want
data = m.query(criteria,properties) # then submit the query using the API

print('Number of compounds of {0} in the Materials Project:  {1}'.format(favourite_formula, len(data)))

We can print out all the data using some simple for loops. The first iterates through each dictionary in the list that is returned by the API. The second iterates through each `key:value` pair in each dictionary:

In [ ]:
for entry in data:
    for key, value in entry.items():
        print('{0} :   {1}'.format(key, value))
    print('-----')

Look again at the form of the `MPRester query` function two cells above. It takes two arguments:   
*Criteria:*  The syntax for this can get a little complex, but is essentially based on Python dictionaries. It allows us to do advanced, specific searches.    
*Properties:*  This is the list of properties which must come from a [predefined list](https://github.com/materialsproject/mapidoc/tree/master/materials) of keywords. These keywords can be used to specify criteria.

The cell below retrieves all the binary oxides of Li and Na. Read it and run it to see what it does: 

In [ ]:
# Getting all the binary oxides of Li and Na in a single search
criteria = {"elements":{"$in":["Li", "Na"], "$all": ["O"]}, "nelements":2}
properties = ['task_id', 'pretty_formula', 'full_formula']
data = m.query(criteria,properties)

print('MP I.D.         full formula')
for entry in data:
    print('{0:15}   {1}'.format(entry['task_id'], entry['full_formula']))

<div class="alert alert-success"> 3. a) Modify the cell above so that it instead retrieves all the binary *nitrides* of Li, Na *and K*    
 b) In addition to the MP I.D. and full formula, have it print out the D.O.I. of the material <br/>

<div class="alert alert-success"> 4. See if you can come up with a query that returns the `'pretty formula'` of every entry in the Materials Project (just for fun) <br/>

In [22]:
# Hint: Think of a property that is common to all the structures in the database...


### Extension exercises

** These exercises are to be carried out in the final cell of the 'Mapping out the chemical space' section **

<div class="alert alert-success"> Extension 1. a) Modify the code so it counts ternary charge neutral combinations instead of binary <br/>

<div class="alert alert-success"> Extension 1. b) Make it print out the total number of ternary combinations *and* the total number of charge neutral ternary combinations in a clear format <br/>

<div class="alert alert-success"> Extension 2. a) Experiment with the neutral_ratios 'threshold' to see how it affects the number of combinations. <br/>

<div class="alert alert-success"> Extension 2. b) Experiment with the number of elements we consider in our initial list at the begining of this section to see how that affects the number of combinations. <br/>